In [83]:
import numpy
import theano
import theano.tensor as T
import pandas as pd
import numpy as np
import sklearn
import sklearn.preprocessing as skp
import sklearn.metrics as skm

def sigmoid(a):
    return 1 / (1 + np.exp(-a))

def sigmoid_derivative(a):
    return sigmoid(a) * (1 - sigmoid(a))

In [84]:
df = pd.read_csv("adult_encoded.data")

In [85]:
df.head()

,Unnamed: 0,age,workclass,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,0,39,3,13,2,8,3,0,1,2174,0,40,0,0
1,1,50,4,13,0,4,2,0,1,0,0,13,0,0
2,2,38,2,9,1,6,3,0,1,0,0,40,0,0
3,3,53,2,7,0,6,2,4,1,0,0,40,0,0
4,4,28,2,13,0,5,0,4,0,0,0,40,12,0


In [86]:
# Random column not reall y sure what it's doing so #dropt
df.drop("Unnamed: 0", axis=1, inplace=True)

y = df.loc[:, df.columns == "salary"]
y_split = np.array_split(y, 2)
y_train = y_split[0]
y_test = y_split[1]

X = df.loc[:, df.columns != "salary"]
X_split = np.array_split(X, 2)
X_train = X_split[0]
X_test = X_split[1]

In [124]:
# first index = number of attributes
# second index = number of neauron in the previous hidden layer

W10 = np.random.randn(12, 4)
W11 = np.random.randn(12, 4)
W20 = np.random.randn(4, 4)
W21 = np.random.randn(4, 4)
W30 = np.random.randn(4, 1)
W31 = np.random.randn(4, 1)

model = {'W10': W10, 'W11' : W11, 'W20' : W20, 'W21' : W21, 'W30' : W30, 'W31' : W31}

# gradient = cost (obj function)
# wrt = X's

# gradient(objective, wrt)

def mu(y, x, W):
    return y * sigmoid(x.dot(W)) + (1 - y) * (1 - sigmoid(x.dot(W)))


In [125]:
# predictor = mu
# obj = modifying weights/mu to minimize cross entropy + complexity, + balance classes


# TODO this is not taking s into account - would need to set num of W's based on unique options
# currently only works for gender
def obj(x, y, W, lambd, eta):
    # sklearn.metrics.log_loss(y_true, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None
    y_t = sigmoid(x.dot(W))
    #cross_entropy = skm.log_loss(y, y_t, normalize=False)
    cross_entropy = -np.log(y * y_t + (1 - y) * (1 - y_t))
    
    # this is manual
    if x[7] == 0:
        prs = df.loc[((df.salary == 0) if y == 0 else (df.salary == 1)) & (df.sex == 0)].shape[0] / df.loc[(df.sex == 0)].shape[0]
        pr = df.loc[((df.salary == 0) if y == 0 else (df.salary == 1))].shape[0] / df.shape[0]
        balance = mu(y, x, W) * np.log((prs) / (pr))
    else:
        prs = df.loc[((df.salary == 0) if y == 0 else (df.salary == 1)) & (df.sex == 1)].shape[0] / df.loc[(df.sex == 1)].shape[0]
        pr = df.loc[((df.salary == 0) if y == 0 else (df.salary == 1))].shape[0] / df.shape[0]
        balance = mu(y, x, W) * np.log((prs) / (pr))
    
    # axis = 1 sample is indepent, axis = 0 feature is independent
    complexity = (W**2).sum()
    
    return cross_entropy + eta * balance + lambd/2. * complexity
        

In [134]:
def predict(x, y, model, lambd, eta):
    W10, W11, W20, W21, W30, W31 = model['W10'], model['W11'], model['W20'], model['W21'], model['W30'], model['W31']
    Z1 = obj(x, y, W10 if x[7] == 0 else W11, lambd, eta)
    print(Z1)
    A1 = mu(y, Z1.transpose(), W10 if x[7] == 0 else W11)
    Z2 = A1.dot(W20 if x[7] == 0 else W21)
    A2 = mu(y, Z2, W20 if x[7] == 0 else W21)
    Z3 = A2.dot(W30 if x[7] == 0 else W31)
    A3 = mu(y, Z2, W30 if x[7] == 0 else W31)
    return A3


In [135]:
predict(X_train.iloc[0].values, y_test.iloc[0].values, model, 1, 0.1)

[         inf  19.02014476          inf  19.02014476]


ValueError: shapes (4,) and (12,4) not aligned: 4 (dim 0) != 12 (dim 0)